In [1]:
#@title Libraries
#General
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data Processing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# Model(ML)
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression

#Framework (DL)
import torch
from torch import nn

# Evaluation
from sklearn.metrics import classification_report # (Actual , prediction)
from sklearn.metrics import accuracy_score # (Actual , Prediction)
from sklearn.metrics import mean_absolute_error #(y_true, y_pred)
from sklearn.metrics import mean_squared_error #(y_true, y_pred)
from sklearn.metrics import r2_score # (y_true, y_pred)

# Imputation
from sklearn.impute import SimpleImputer

#Dataset
from sklearn.datasets import make_circles           #X ,  y = make_circles( n_samples, noise = 0.03 , random_state = 42)


In [2]:
train =  pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test =  pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [3]:
X = train.drop(['id',"FloodProbability"] , axis = 1)
y =  train["FloodProbability"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# #-------------------------Scalling----------------------------------
# scaler = StandardScaler(  )
# X_train = scaler.fit_transform( X_train )
# X_test =  scaler.transform( X_test )





#---------------------Convert to Tensor-----------------------------
X_train = torch.from_numpy( X_train.to_numpy() ).type( torch.float ).to("cuda")
y_train  = torch.from_numpy( y_train.to_numpy() ).type( torch.float ).to("cuda")
X_test =  torch.from_numpy( X_test.to_numpy() ).type( torch.float ).to("cuda")
y_test =  torch.from_numpy( y_test.to_numpy() ) .type( torch.float ).to("cuda")

print("---------------Scalling & Tensor Conversion Done------------")

---------------Scalling & Tensor Conversion Done------------


In [5]:
class AutoEncoder( nn.Module ):
  def __init__(self , input_dimentions ):
    super().__init__()
    self.encoder  = nn.Sequential(
        
        nn.Linear( in_features = input_dimentions , out_features = 15),
        nn.ReLU(),
        nn.Linear( in_features = 15, out_features = 10),
        
        
    ) 
    self.decoder = nn.Sequential(
        
        nn.Linear( in_features = 10 , out_features = 15),
        nn.ReLU(),
        nn.Linear( in_features = 15 , out_features = input_dimentions )
        
    )

  def forward( self , x ):
    return self.decoder( self.encoder( x ) )



#Actual Model
Encoder  = AutoEncoder( 20 ).to('cuda')


optimizer = torch.optim.SGD( params = Encoder.parameters() , lr = 0.01 )
loss_fn  = nn.MSELoss()


for epoch in range( 10000 ):

  Encoder.train()
  y_pred = Encoder( X_train )

  loss = loss_fn(   y_pred , X_train   )

  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 100 == 0 :
    Encoder.eval()
    with torch.inference_mode():
      y_pred_test =  Encoder( X_test )
     
      print( "Epoch:" , epoch , "MAE:" , mean_absolute_error( X_test.cpu(), y_pred_test.cpu() ) , end = ' ')
      print( "Loss:" , loss )

Epoch: 0 MAE: 4.86332 Loss: tensor(28.5227, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 100 MAE: 1.6148603 Loss: tensor(4.1358, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 200 MAE: 1.6040853 Loss: tensor(4.0773, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 300 MAE: 1.5938947 Loss: tensor(4.0133, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 400 MAE: 1.5878742 Loss: tensor(3.9475, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 500 MAE: 1.564286 Loss: tensor(3.8260, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 600 MAE: 1.5364395 Loss: tensor(3.7061, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 700 MAE: 1.5062853 Loss: tensor(3.6009, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 800 MAE: 1.4861152 Loss: tensor(3.4986, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 900 MAE: 1.4546026 Loss: tensor(3.3561, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 1000 MAE: 1.4210671 Loss: tensor(3.2086, device='cuda:0', grad_fn=<MseLossBac

**Model Training**

In [6]:

# #using Sklearn
# regressor = LinearRegression().fit(X_train,y_train)
# regressor.score( X_test , y_test )
# R2 Score Acheived : 0.8448773362840329

In [7]:
class Model( nn.Module ):
  def __init__(self , input_dimentions  , output_dimentions , encoder ):
    super().__init__()
    self.encoder =  encoder 
    self.block1  = nn.Sequential(
        
        nn.Linear( in_features = input_dimentions , out_features = 5),
        nn.ReLU(),
        nn.Linear( in_features = 5 , out_features = 1)
        
    ) 

  def forward( self , x ):
    return self.block1( self.encoder( x ) )

In [8]:
Encoder.eval()

device = 'cuda'  #'gpu'
learning_rate = 0.01
total_epoch =  18000

#Actual Model
model  = Model( 20 , 1  , Encoder).to(device)


optimizer = torch.optim.SGD( params = model.parameters() , lr = learning_rate )
loss_fn  = nn.MSELoss()


#training Loop
for epoch in range( total_epoch ):

  model.train()
  y_pred = model( X_train )


  loss = loss_fn(   y_pred.squeeze() , y_train   )
#   print( loss )   
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 100 == 0 :
    model.eval()
    with torch.inference_mode():
      y_pred_test =  model( X_test ).cpu().squeeze()
     
      print( "MAE:" , mean_absolute_error( y_test.cpu() , y_pred_test ) , end = ' ')
      print( "R2:" , r2_score( y_test.cpu() , y_pred_test) )


MAE: 0.31481096 R2: -39.41722227663259
MAE: 0.06272002 R2: -1.3814537197878214
MAE: 0.045202646 R2: -0.23040783025980405
MAE: 0.040514212 R2: 0.017643319409673675
MAE: 0.03841626 R2: 0.12056860939341318
MAE: 0.037222747 R2: 0.17700639939462492
MAE: 0.03644072 R2: 0.21310298159223717
MAE: 0.03587755 R2: 0.23861644691881723
MAE: 0.03544402 R2: 0.2580047751342499
MAE: 0.03509213 R2: 0.2735772897998936
MAE: 0.034794137 R2: 0.28663665540648375
MAE: 0.03453353 R2: 0.29796304849063293
MAE: 0.03429913 R2: 0.3080622762167994
MAE: 0.034083806 R2: 0.3172682872218334
MAE: 0.03388265 R2: 0.3258091823139896
MAE: 0.033691946 R2: 0.33384536280623944
MAE: 0.033509128 R2: 0.34149136740182984
MAE: 0.03333231 R2: 0.3488318423004705
MAE: 0.03316023 R2: 0.355927973137132
MAE: 0.032991834 R2: 0.3628269686869199
MAE: 0.032826245 R2: 0.3695643362112001
MAE: 0.032662857 R2: 0.3761683769025713
MAE: 0.032501213 R2: 0.38266125879773205
MAE: 0.032340925 R2: 0.3890597365222319
MAE: 0.0321817 R2: 0.39537617775991707


In [9]:
test.shape

(745305, 21)

In [10]:
X_test = torch.from_numpy( test.drop('id' , axis =  1).to_numpy() ).type( torch.float ).to('cuda')

In [11]:
model.eval()
with torch.inference_mode():
    prediction =  model( X_test )

In [12]:
prediction = prediction.squeeze().cpu()

In [13]:
submission = pd.DataFrame(
    {
    'id': test["id"],
    'FloodProbabiliy': prediction
    }
)
submission.to_csv("submission.csv" , index = False)

In [14]:
submission.to_csv("submission.csv" , index = False)

In [15]:
submission

,id,FloodProbabiliy
0,1117957,0.567205
1,1117958,0.460423
2,1117959,0.459492
3,1117960,0.471139
4,1117961,0.467940
...,...,...
745300,1863257,0.480418
745301,1863258,0.453272
745302,1863259,0.612207
745303,1863260,0.545476
